In this notebook, we demonstrate how EnergyPlus can be used as a library within Python, so that a running simulation can be controlled by a Python script

Pre-requisites: 
1. EnergyPlus v22.2 (any version >9.6 should work, must have Python API)
2. An idf file describing the building

In [1]:
# Import the EnergyPlus Library
import sys, os
from shutil import rmtree
sys.path.insert(0, 'E:/EnergyPlusV22-2-0/') # Point to E+ installation directory

from pyenergyplus.api import EnergyPlusAPI
api = EnergyPlusAPI() # This handler now co-ordinates all the E+ related stuff

import pandas as pd # For reporting
from numpy.random import normal

In [2]:
# Setup the inputs and directories
idf_file = os.path.realpath("./Example Building/AdultEducationCenter.idf") # 1-zone building with an ideal loads air system
weather_file = os.path.realpath("./Example Building/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw")

output_dir = os.path.join(os.getcwd(),"DemonstrationOutputs")
rmtree(output_dir, ignore_errors=True)

In [3]:
# Define callbacks which only do reporting (ie, no modifications)
class NoModificationCallbacks:
    def __init__(self, api, state, output_dir, debug=True):
        self.out_dir = output_dir
        self.api = api
        self.state = state
        self.debug = debug
        self.get_handles(state)
        self.data = {kk:[] for kk in self.handles}
        self.data.update({'CurrTime':[], 'DayOfYear':[]})
        self.register_callbacks()
        
    # Zone,Average,Zone Mean Air Temperature [C]    
    def get_handles(self, state):
        # Get handles to the outdoor temperature, the thermostat setpoint, Electricity, Heating and Cooling
        s = state
        # Variables can be looked up from the .rdd file, meters from .mdd and actuators from .edd
        self.handles = dict(
            OutdoorTemp=self.api.exchange.get_variable_handle(s, "Site Outdoor Air Drybulb Temperature", "Environment"),
            ZoneTemp=self.api.exchange.get_variable_handle(s, "Zone Air Temperature", "FULL BUILDING - 1 ZONE"),
            ThermostatSetpoint=self.api.exchange.get_actuator_handle(s, "System Node Setpoint", "Temperature Setpoint", "FULL BUILDING - 1 ZONE ZONE AIR NODE"),
            ElectricityMeter=self.api.exchange.get_meter_handle(s, "Electricity:Building"),
            HeatingLoad=self.api.exchange.get_meter_handle(s, "DistrictHeating:Facility"),
            CoolingLoad=self.api.exchange.get_meter_handle(s, "DistrictCooling:Facility"),
        )
        if self.debug:
            pass
        #    print("==Handles==\n\n", self.handles)

    def on_end_step(self, state):
        # In this example, we query some parameters from the simulation at every time step
        # Make sure warm-up is complete
        if self.api.exchange.warmup_flag(state):
            return
        self.get_handles(state)
        s = state
        self.data["CurrTime"].append(self.api.exchange.current_time(s))
        self.data["DayOfYear"].append(self.api.exchange.day_of_year(s))
        self.data["OutdoorTemp"].append(self.api.exchange.get_variable_value(s, self.handles["OutdoorTemp"]))
        self.data["ZoneTemp"].append(self.api.exchange.get_variable_value(s, self.handles["ZoneTemp"]))
        self.data["ThermostatSetpoint"].append(self.api.exchange.get_actuator_value(s, self.handles["ThermostatSetpoint"]))
        self.data["ElectricityMeter"].append(self.api.exchange.get_meter_value(s, self.handles["ElectricityMeter"]))
        self.data["HeatingLoad"].append(self.api.exchange.get_meter_value(s, self.handles["HeatingLoad"]))
        self.data["CoolingLoad"].append(self.api.exchange.get_meter_value(s, self.handles["CoolingLoad"]))

    def on_end_sim(self):
        # We consolidate the data into a dataframe and write a csv output
        df = pd.DataFrame.from_dict(self.data)
        df.to_excel(os.path.join(self.out_dir, "CustomReport.xlsx"))

    def register_callbacks(self):
        # Here, we register the callbacks so that the API calls them at the appropriate time
        self.api.runtime.callback_end_system_timestep_after_hvac_reporting(self.state, self.on_end_step)

In [4]:
# Make a run without any modifications
output_dir_no_modifications = os.path.join(output_dir, "NoModifications")

state = api.state_manager.new_state() 
api.state_manager.reset_state(state)
api.runtime.set_console_output_status(state,1) # Generates .err files for debug

# Register Callbacks for no modifications run
callbacks = NoModificationCallbacks(api, state, output_dir_no_modifications)

run_status = api.runtime.run_energyplus(state, ["-d", output_dir_no_modifications, "-w", weather_file, idf_file])

print("Unmodified run completed with status", run_status)
callbacks.on_end_sim()

Unmodified run completed with status 0


In [5]:
# Now, define the callbacks as per requirement
# For example, here we set the value of the temperature setpoint to be equal to the time of day
class WithModificationCallbacks:
    def __init__(self, api, state, output_dir, debug=True):
        self.out_dir = output_dir
        self.api = api
        self.state = state
        self.debug = debug
        self.printonce = True
        self.get_handles(state)
        self.data = {kk:[] for kk in self.handles}
        self.data.update({'CurrTime':[], 'DayOfYear':[]})
        self.register_callbacks()
        
    def get_handles(self, state):
        # Get handles to the outdoor temperature, the thermostat setpoint, Electricity, Heating and Cooling
        s = state
        # Variables can be looked up from the .rdd file, meters from .mdd and actuators from .edd
        self.handles = dict(
            OutdoorTemp=self.api.exchange.get_variable_handle(s, "Site Outdoor Air Drybulb Temperature", "Environment"),
            ZoneTemp=self.api.exchange.get_variable_handle(s, "Zone Air Temperature", "FULL BUILDING - 1 ZONE"),
            ThermostatSetpoint=self.api.exchange.get_actuator_handle(s, "System Node Setpoint", "Temperature Setpoint", "FULL BUILDING - 1 ZONE ZONE AIR NODE"),
            ElectricityMeter=self.api.exchange.get_meter_handle(s, "Electricity:Building"),
            HeatingLoad=self.api.exchange.get_meter_handle(s, "DistrictHeating:Facility"),
            CoolingLoad=self.api.exchange.get_meter_handle(s, "DistrictCooling:Facility"),
        )
        if self.debug:
            pass
        #    print("==Handles==\n\n", self.handles)

    def on_start_step(self, state):
        # Need to do it this way, as Node temperatures cannot be changed using set_actuator_value function (apparently)
        hot_thermostat = self.api.exchange.get_actuator_handle(state, "Zone Temperature Control", "Heating Setpoint", "FULL BUILDING - 1 ZONE")
        cool_thermostat = self.api.exchange.get_actuator_handle(state, "Zone Temperature Control", "Cooling Setpoint", "FULL BUILDING - 1 ZONE")
        # Induce a sudden change in the setpoint
        current_time = self.api.exchange.current_time(state)
        next_setpoint_delta = 0 if current_time < 12 else -10
        self.api.exchange.set_actuator_value(state, hot_thermostat, 15.0 + next_setpoint_delta)
        self.api.exchange.set_actuator_value(state, cool_thermostat, 25.0 + next_setpoint_delta)

    def on_end_step(self, state):
        # In this example, we query some parameters from the simulation at every time step
        if self.api.exchange.warmup_flag(state):
            return
        self.get_handles(state)
        s = state
        self.data["CurrTime"].append(self.api.exchange.current_time(s))
        self.data["DayOfYear"].append(self.api.exchange.day_of_year(s))
        self.data["OutdoorTemp"].append(self.api.exchange.get_variable_value(s, self.handles["OutdoorTemp"]))
        self.data["ZoneTemp"].append(self.api.exchange.get_variable_value(s, self.handles["ZoneTemp"]))
        self.data["ThermostatSetpoint"].append(self.api.exchange.get_actuator_value(s, self.handles["ThermostatSetpoint"]))
        self.data["ElectricityMeter"].append(self.api.exchange.get_meter_value(s, self.handles["ElectricityMeter"]))
        self.data["HeatingLoad"].append(self.api.exchange.get_meter_value(s, self.handles["HeatingLoad"]))
        self.data["CoolingLoad"].append(self.api.exchange.get_meter_value(s, self.handles["CoolingLoad"]))

    def on_end_sim(self): # For now, this is called manually
        # We consolidate the data into a dataframe and write a csv output
        df = pd.DataFrame.from_dict(self.data)
        df.to_excel(os.path.join(self.out_dir, "CustomReport.xlsx"))

    def register_callbacks(self):
        # Here, we register the callbacks so that the API calls them at the appropriate time
        #self.api.runtime.callback_begin_system_timestep_before_predictor(self.state, self.on_start_step)
        self.api.runtime.callback_after_predictor_before_hvac_managers(self.state, self.on_start_step)
        self.api.runtime.callback_end_system_timestep_after_hvac_reporting(self.state, self.on_end_step)

In [6]:
# Make a run with modifications
output_dir_with_modifications = os.path.join(output_dir, "WithModifications")

state = api.state_manager.new_state() 
api.state_manager.reset_state(state)
api.runtime.set_console_output_status(state,1) # Generates .err files for debug

# Register Callbacks for no modifications run
callbacks = WithModificationCallbacks(api, state, output_dir_with_modifications)

run_status = api.runtime.run_energyplus(state, ["-d", output_dir_with_modifications, "-w", weather_file, idf_file])

print("Modified run completed with status", run_status)
callbacks.on_end_sim()

Modified run completed with status 0
